In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
import timeit

In [2]:
Data = pd.read_excel('Data diolah darin.xlsx')

In [3]:
Data1 = Data.copy()
Data1.replace('#DIV/0!',np.nan,inplace=True)
Data1.dropna(inplace=True)

X = Data1[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = Data1['HC[g/s]']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_9864\577416567.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data1.replace('#DIV/0!',np.nan,inplace=True)


In [4]:
br = BaggingRegressor(estimator=LinearRegression(), 
                      n_estimators=10,
                      max_samples=1.0, 
                      max_features=1.0, 
                      bootstrap=True, 
                      bootstrap_features=False, 
                      oob_score=False)
br.fit(X_train, y_train.values.ravel())
y_pred = br.predict(X_train)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

Mean Absolute Error: 0.0011304053159406587
Mean Squared Error: 3.858830849193943e-06
Root Mean Squared Error: 0.0019643907068589853


## Genetic Algorithm

In [5]:
import timeit
start = timeit.default_timer() # catat waktu mulai

# GA
from pyMetaheuristic.algorithm import genetic_algorithm

# Define parameters to be tuned and their ranges
X = X_train
y = y_train

def create_fitness_function(X, y):
    def fitness_function(variables):
        n_estimators = int(round(variables[0]))
        max_samples = variables[1]
        max_features = variables[2]
        bootstrap_map = {0: 'True', 1: 'False'}
        bootstrap_index = int(np.clip(variables[3], 0, 1))
        bootstrap = bootstrap_map[bootstrap_index] == 'True'  # Convert to boolean
        bootstrap_features_map = {0: 'True', 1: 'False'}
        bootstrap_features_index = int(np.clip(variables[4], 0, 1))
        bootstrap_features = bootstrap_features_map[bootstrap_features_index] == 'True'  # Convert to boolean
        oob_score = bootstrap  # Set oob_score to True only when bootstrap is True
        
        br_tune = BaggingRegressor(estimator=LinearRegression(), n_estimators=n_estimators, max_samples=max_samples, max_features=max_features, 
                                   bootstrap=bootstrap, bootstrap_features=bootstrap_features, oob_score=oob_score)
        br_tune.fit(X, y)
        y_pred = br_tune.predict(X)
        scores = np.sqrt(mean_squared_error(y, y_pred))
        return scores
    return fitness_function

fitness_function = create_fitness_function(X, y)

# GA - Parameters
parameters = {
    'population_size': 100,
    'min_values': [10, 0.5, 0.5, 0, 0, 0],  # min values for n_estimators, max_samples, and max_features, bootstrap, bootstrap_features, oob_score
    'max_values': [1000, 1, 1, 2, 2, 2],   # max values for  n_estimators, max_samples, and max_features, bootstrap, bootstrap_features, oob_score
    'generations': 1000,
    'mutation_rate': 0.1,
    'elite': 1,
    'eta': 1,
    'mu': 1,
    'verbose': True,
    'start_init': None,
    'target_value': None
}

# GA - Algorithm
ga = genetic_algorithm(target_function = fitness_function, **parameters)

# GA - Solution
variables = ga[:-1]
minimum   = ga[ -1]
print('Variables: ', np.around(variables, 4) , ' Minimum Value Found: ', round(minimum, 4) )


stop = timeit.default_timer() # catat waktu selesai
lama_eksekusi = stop - start # lama eksekusi dalam satuan detik
print("Lama eksekusi: ",lama_eksekusi,"detik")

Generation =  0  f(x) =  0.0019653762511094445


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some in

Generation =  1  f(x) =  0.001964376046794619
Generation =  2  f(x) =  0.0019643758565724093


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some in

Generation =  3  f(x) =  0.001964375838320711


...

Generation =  999  f(x) =  0.0019643758042112093


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some in

Generation =  1000  f(x) =  0.0019643758042112093


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:1225: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(


Variables:  [5.164132e+02 1.000000e+00 1.000000e+00 1.946100e+00 1.996800e+00
 2.645000e-01]  Minimum Value Found:  0.002
Lama eksekusi:  408293.4867304 detik


In [6]:
br = BaggingRegressor(estimator=LinearRegression(), 
                      n_estimators=472,
                      max_samples=1.0, 
                      max_features=1.0, 
                      bootstrap=False, 
                      bootstrap_features=False, 
                      oob_score=False)
br.fit(X_train, y_train.values.ravel())
y_pred = br.predict(X_train)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

Mean Absolute Error: 0.001131364181519164
Mean Squared Error: 3.858772300170436e-06
Root Mean Squared Error: 0.0019643758042112093
